In [1]:
import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()
import numpy as np
import random
import gym
from gym.envs.registration import register
from argparse import ArgumentParser


In [2]:
class Environment():
    def __init__(self,observation_space_n,action_space_n,state_space,reward_space,state_space_width,state_space_length,action_space):
        self.observation_space_n=observation_space_n
        self.action_space_n=action_space_n
        self.state_space=state_space
        self.reward_space=reward_space
        self.state_space_width=state_space_width
        self.state_space_length=state_space_length
        self.action_space=action_space

    def __del__(self):
        pass

class DeepQAgent():
    def __init__(self, env):
        # set hyperparameters
        self.max_episodes = 1000
        self.max_actions = 99
        self.discount = 0.93
        self.exploration_rate = 1.0
        self.exploration_decay = 1.0/self.max_episodes
        # get envirionment
        self.env = env
    
        # nn_model parameters
        self.in_units = env.observation_space_n
        self.out_units = env.action_space_n
        self.hidden_units = 10
        
        # construct nn model
        self._nn_model()
    
        # save nn model
        self.saver = tf.train.Saver()
    def _nn_model(self):
        self.a0 = tf.placeholder(tf.float32, shape=[1, self.in_units]) # input layer
        self.y = tf.placeholder(tf.float32, shape=[1, self.out_units]) # ouput layer
        
        # from input layer to hidden layer
        self.w1 = tf.Variable(tf.zeros([self.in_units, self.hidden_units], dtype=tf.float32)) # weight
        self.b1 = tf.Variable(tf.random.uniform([self.hidden_units], 0, 0.01, dtype=tf.float32)) # bias
        self.a1 = tf.nn.relu(tf.matmul(self.a0, self.w1) + self.b1) # the ouput of hidden layer
        
        # from hidden layer to output layer
        self.w2 = tf.Variable(tf.zeros([self.hidden_units, self.out_units], dtype=tf.float32)) # weight
        self.b2 = tf.Variable(tf.random.uniform([self.out_units], 0, 0.01, dtype=tf.float32)) # bias
        
        # Q-value and Action
        self.a2 = tf.matmul(self.a1, self.w2) + self.b2 # the predicted_y (Q-value) of four actions
        self.action = tf.argmax(self.a2, 1) # the agent would take the action which has maximum Q-value
        print(self.action[0])
        # loss function
        self.loss = tf.reduce_sum(tf.square(self.a2-self.y))
        
        # upate model
        self.update_model =  tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(self.loss)
    def train(self,env,real=True):
        # get hyper parameters
        max_episodes = self.max_episodes
        if(not real):
            max_episodes=1
        max_actions = self.max_actions
        discount = self.discount
        exploration_rate = self.exploration_rate
        exploration_decay = self.exploration_decay
        
        # start training
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer()) # initialize tf variables
            for i in range(max_episodes):
                print(i,i/max_episodes)
                # state = env.reset()[0] # reset the environment per eisodes
                state = env.state_space.index('S')
                for j in range(max_actions):
                     # get action and Q-values of all actions
                    # print(self.action)
                    self.action = tf.argmax(self.a2, 1)
                    action, pred_Q = sess.run([self.action, self.a2],feed_dict={self.a0:np.eye(self.in_units)[state:state+1]})
                    actionn=action[0]
                    # if explorating, then taking a random action instead
                    if np.random.rand()<exploration_rate: 
                        # action[0] = env.action_space.sample()
                        actionn = random.randint(0,env.action_space[-1])
                        action[0]=actionn
                    # print(action)
                    # print(state,actionn,end=' | ')
                    # get nextQ in given next_state
                    state_x=state%env.state_space_width
                    state_y=state//env.state_space_width
                    if(actionn==0):
                        if (state_y>0):
                            state_y-=1
                    elif(actionn==1):
                        if (state_x<env.state_space_width-1):
                            state_x+=1
                    elif(actionn==2):
                        if (state_y<env.state_space_length-1):
                            state_y+=1
                    elif(actionn==3):
                        if (state_x>0):
                            state_x-=1
                    next_state=state_x+state_y*env.state_space_width
                    rewards=env.reward_space[next_state]
                    if(env.state_space[next_state]=='G' or env.state_space[next_state]=='W'):
                        done=True
                    else:
                        done=False
                    # next_state, rewards, done, terminated, info = env.step(action[0])
                    next_Q = sess.run(self.a2,feed_dict={self.a0:np.eye(self.in_units)[next_state:next_state+1]})
                    # update
                    update_Q = pred_Q
                    print(update_Q,state,actionn,action)
                    update_Q [0,action] = rewards + discount*np.max(next_Q)
                    sess.run([self.update_model],
                             feed_dict={self.a0:np.identity(self.in_units)[state:state+1],self.y:update_Q})
                    state = next_state
                    # if fall in the hole or arrive to the goal, then this episode is terminated.
                    if done:
                        if exploration_rate > 0.001:
                            exploration_rate -= exploration_decay
                        break
            # save model
            if(real):
                save_path = self.saver.save(sess, "./nn_model.ckpt")
    def test(self,env):
        # get hyper-parameters
        max_actions = self.max_actions
        # start testing
        with tf.Session() as sess:
            # restore the model
            sess.run(tf.global_variables_initializer())
            saver=tf.train.import_meta_graph("./nn_model.ckpt.meta") # restore model
            saver.restore(sess, tf.train.latest_checkpoint('./'))# restore variables
            
            # testing result
            # state = env.reset()
            state = env.state_space.index('S')
            for j in range(max_actions):
                # env.render() # show the environments
                # always take optimal action
                # for i in range(16):
                #     state=i
                action, pred_Q = sess.run([self.action, self.a2],feed_dict={self.a0:np.eye(self.in_units)[state:state+1]})
                print(pred_Q)
                # break
                # update
                actionn=action[0]
                print(state,actionn)
                state_x=state%env.state_space_width
                state_y=state//env.state_space_width
                if(actionn==0):
                    if (state_y>0):
                        state_y-=1
                elif(actionn==1):
                    if (state_x<env.state_space_width-1):
                        state_x+=1
                elif(actionn==2):
                    if (state_y<env.state_space_length-1):
                        state_y+=1
                elif(actionn==3):
                    if (state_x>0):
                        state_x-=1
                next_state=state_x+state_y*env.state_space_width
                rewards=env.reward_space[next_state]
                if(env.state_space[next_state]=='G' or env.state_space[next_state]=='W'):
                    done=True
                else:
                    done=False
                # next_state, rewards, done, info = env.step(action[0])
                state = next_state
                if done:
                    # env.render()
                    break
    def displayQ():
        # show information
        pass

In [3]:
# env = gym.make('FrozenLake8x8-v1')
action_space = [0,1,2,3]#up,right,down,left,nothing
state_space_width=20 # S is start, T is tile, W is Wall, G is Goal
state_space_length=20
state_space = ['T','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','W',
               'T','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','G',
               'T','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','W','T','T',
               'T','T','T','T','T','T','T','W','W','T','W','T','T','T','T','T','W','T','T','T',
               'T','T','T','T','T','T','W','T','T','T','W','T','T','T','T','W','T','T','T','T',
               'T','T','T','T','T','T','W','T','W','T','W','T','T','T','W','T','T','T','T','T',
               'T','T','T','T','T','T','W','T','W','T','W','T','T','W','T','T','T','T','T','T',
               'T','W','T','T','T','T','W','T','W','T','W','T','W','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','W','T','T','W','T','T','T','T','T','T','T','T',
               'S','W','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'T','W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T',
               'W','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T','T']
state_space_width=4 # S is start, T is tile, W is Wall, G is Goal
state_space_length=4
state_space = ['S','T','T','T',
               'T','T','T','T',
               'T','W','W','T',
               'T','T','T','G']
# for i in range(len(state_space)):
#     if not (state_space[i]=='S' or state_space[i]=='G'):
#         if(random.randint(0,8)==0):
#             state_space[i]='W'
#         else:
#             state_space[i]='T'
# print(len(state_space))
reward_space = []
for i in state_space:
    if(i=='S'):
        reward_space.append(-1)
    elif(i=='G'):
        reward_space.append(10)
    elif(i=='T'):
        reward_space.append(-1)
    elif(i=='W'):
        reward_space.append(-100)
# print('  Reward matrix')
# for i in range(len(reward_space)):
#     print('{:4}'.format(reward_space[i]),end = ',')
#     if(i%state_space_width==state_space_width-1):
#         print()

observation_space_n=len(state_space)
action_space_n=len(action_space)
state_space
reward_space
state_space_width
state_space_length
if __name__ == '__main__':
    env=Environment(observation_space_n,action_space_n,state_space,reward_space,state_space_width,state_space_length,action_space)
    agent = DeepQAgent(env) # get agent


Tensor("strided_slice:0", shape=(), dtype=int64)


In [4]:
if __name__ == '__main__':
    agent.train(env)
    # agent.train(env,False)
    print("\n==============TEST==============\n")


0 0.0
[[0.00951366 0.00677337 0.00348903 0.00607125]] 0 3 [3]
[[ 0.00951366  0.00677337  0.00348903 -0.09367686]] 0 1 [1]
[[ 0.00951366 -0.09304496  0.00348903 -0.09367686]] 1 3 [3]
[[ 0.00951366 -0.09304542  0.00348903 -0.18344845]] 1 2 [2]
[[ 0.00951366 -0.09304519 -0.09600148 -0.183448  ]] 0 2 [2]
[[ 0.00951366 -0.09304566 -0.18554099 -0.1834489 ]] 0 2 [2]
[[ 0.00951366 -0.09304646 -0.26612625 -0.1834504 ]] 0 2 [2]
[[ 0.00951366 -0.09304748 -0.33865342 -0.18345235]] 0 1 [1]
[[ 0.00951366 -0.18288454 -0.33865103 -0.18345106]] 1 2 [2]
[[ 0.00951366 -0.182888   -0.4039278  -0.18345435]] 0 0 [0]
[[-0.09058486 -0.18288526 -0.403922   -0.18345174]] 4 1 [1]
[[-0.09058538 -0.27304992 -0.40392393 -0.1834526 ]] 5 0 [0]
1 0.001
[[-10.096524    -0.27314112  -0.40405354  -0.18351096]] 0 3 [3]
[[-10.096632    -0.27314398  -0.40405762  -0.2823647 ]] 0 0 [0]
[[-9.205658   -0.27296555 -0.40380406 -0.28211406]] 4 2 [2]
[[-9.205658   -0.27296555 -0.48880947 -0.28211406]] 4 0 [0]
[[-8.410478   -0.27296

In [ ]:
if __name__ == '__main__':
    agent.test(env)

INFO:tensorflow:Restoring parameters from ./nn_model.ckpt


[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565  -15.730893 -14.285641 -14.28565 ]]
0 2
[[-24.18565 